In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt

# Notebook printing output
from xopt import output_notebook
output_notebook()

In [3]:
import yaml
from xopt import Xopt

YAML="""
xopt:
  output_path: temp

algorithm:
  name: cnsga
  options:
    max_generations: 3
    population_size: 8
    crossover_probability: 0.9
    mutation_probability: 1.0
    selection: auto
    population: null
    show_progress: True

simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options:
    astra_input_file: ../templates/kekgun/kekgun.in
    generator_input_file: ../templates/kekgun/dist004.in

# Note that you can call another file in the top level group:
vocs:
    variables:
      sig_x: [0.05, 1]
      lt: [0.005, 0.07]
      maxe(1): [20, 50]
      phi(1): [-30, 30]
      maxb(1): [0, 0.4]
      maxe(2): [0, 32]
      phi(2): [-180, 180]
      maxb(2): [0, 0.3]
      maxe(3): [0, 32]
      maxe(4): [0, 32]
      phi(3): [-45, 45]
      phi(4): [-45, 45]
      phi(6): [-45, 45]
    constants:
      ipart: 1000
      lspch: true
      zstop: 16.54

    objectives:
      end_core_emit_95percent_x: MINIMIZE
      end_sigma_z: MINIMIZE
    constraints:
      end_sigma_z: [LESS_THAN, 0.0015]
      end_core_emit_95percent_x: [LESS_THAN, 9.0e-07]
      end_sigma_energy: [LESS_THAN, 200000.0]
      end_higher_order_energy_spread: [LESS_THAN, 5000.0]
      end_mean_kinetic_energy: [GREATER_THAN, 90000000.0]
      end_n_particle_loss: [LESS_THAN, 1]
    linked_variables: null

"""

!mkdir temp/

In [4]:
# Create object
X = Xopt(YAML)

# Change some things to make it run faster
X.vocs['constants']['lspch'] = True
X.vocs['constants']['ipart'] = 1000
X.vocs['constants']['zstop'] = 0.2

# Show config
X

Loading config from text
Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.



            Xopt 
________________________________           
Version: 0.4.3+178.g392b747.dirty
Configured: True
Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp}
algorithm:
  name: cnsga
  options: {max_generations: 3, population_size: 8, crossover_probability: 0.9, mutation_probability: 1.0,
    selection: auto, population: null, show_progress: true, seed: null, verbose: null}
  function: xopt.cnsga.cnsga
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in,
    workdir: null, astra_bin: /Users/chrisonian/Code/astra/bin/Astra, generator_bin: /Users/chrisonian/Code/astra/bin/generator,
    timeout: 2500, verbose: false, auto_set_spacecharge_mesh: true, archive_path: null,
    merit_f: null}
  function:

In [5]:
# Check random inputs 
X.random_inputs()

{'sig_x': 0.49714987199980876,
 'lt': 0.02735233801053188,
 'maxe(1)': 36.55600097379309,
 'phi(1)': -5.206366581287233,
 'maxb(1)': 0.12530404687494406,
 'maxe(2)': 1.6812802317750624,
 'phi(2)': 20.646092792752285,
 'maxb(2)': 0.12004300057653916,
 'maxe(3)': 18.281079927992067,
 'maxe(4)': 23.619839156234434,
 'phi(3)': -12.085834669106106,
 'phi(4)': -31.16304430093622,
 'phi(6)': 16.205974634505218,
 'ipart': 1000,
 'lspch': True,
 'zstop': 0.2}

In [6]:
# Evaluate with some particular settings
X.evaluate(
    {'ipart': 1000,
 'lspch': True,
 'zstop': 0.2})

{'error': False,
 'end_mean_z': 0.2,
 'end_norm_emit_x': 3.6377e-07,
 'end_core_emit_95percent_x': 2.464e-07,
 'end_core_emit_90percent_x': 2.0333e-07,
 'end_core_emit_80percent_x': 1.3782e-07,
 'end_norm_emit_y': 3.619e-07,
 'end_core_emit_95percent_y': 2.4596e-07,
 'end_core_emit_90percent_y': 2.0308999999999997e-07,
 'end_core_emit_80percent_y': 1.3819999999999998e-07,
 'end_norm_emit_z': 12.845,
 'end_core_emit_95percent_z': 1.0246e-05,
 'end_core_emit_905percent_z': 8.504300000000001e-06,
 'end_core_emit_80percent_z': 6.086e-06,
 'end_mean_t': 7.1444e-10,
 'end_mean_x': 1.1971e-06,
 'end_sigma_x': 0.00087335,
 'end_sigma_xp': 0.0038271000000000004,
 'end_cov_x__xp': 3.3419611099999997e-06,
 'end_mean_y': 1.9815000000000004e-06,
 'end_sigma_y': 0.0008733800000000001,
 'end_sigma_yp': 0.003828,
 'end_cov_y__yp': 3.3428619500000005e-06,
 'end_mean_kinetic_energy': 2996800.0,
 'end_sigma_z': 0.0023518,
 'end_sigma_energy': 13976.0,
 'end_cov_z__energy': -30.255906999999997,
 'end_n_pa

In [7]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

CPU times: user 340 ms, sys: 26.2 ms, total: 367 ms
Wall time: 9.04 s


{'error': False,
 'end_mean_z': 0.2,
 'end_norm_emit_x': 1.8372e-06,
 'end_core_emit_95percent_x': 1.5259999999999999e-06,
 'end_core_emit_90percent_x': 1.2681e-06,
 'end_core_emit_80percent_x': 8.4845e-07,
 'end_norm_emit_y': 1.7617e-06,
 'end_core_emit_95percent_y': 1.4453e-06,
 'end_core_emit_90percent_y': 1.1918e-06,
 'end_core_emit_80percent_y': 7.7903e-07,
 'end_norm_emit_z': 11.973,
 'end_core_emit_95percent_z': 9.877599999999998e-06,
 'end_core_emit_905percent_z': 8.5207e-06,
 'end_core_emit_80percent_z': 6.8079e-06,
 'end_mean_t': 7.2388e-10,
 'end_mean_x': 9.889200000000002e-06,
 'end_sigma_x': 0.0019256,
 'end_sigma_xp': 0.0087977,
 'end_cov_x__xp': 1.693834784e-05,
 'end_mean_y': 4.8288e-06,
 'end_sigma_y': 0.0018874,
 'end_sigma_yp': 0.008619300000000002,
 'end_cov_y__yp': 1.626580194e-05,
 'end_mean_kinetic_energy': 2749600.0,
 'end_sigma_z': 0.0024345,
 'end_sigma_energy': 13478.0,
 'end_cov_z__energy': -30.5505405,
 'end_n_particle_loss': 453,
 'end_total_charge': 5.47e

In [8]:
# These are the algorithm options
X.algorithm['options']

{'max_generations': 3,
 'population_size': 8,
 'crossover_probability': 0.9,
 'mutation_probability': 1.0,
 'selection': 'auto',
 'population': None,
 'show_progress': True,
 'seed': None,
 'verbose': None}

In [9]:
# These are the options in the evaluate function
X.simulation['options']

{'astra_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in',
 'generator_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in',
 'workdir': None,
 'astra_bin': '/Users/chrisonian/Code/astra/bin/Astra',
 'generator_bin': '/Users/chrisonian/Code/astra/bin/generator',
 'timeout': 2500,
 'verbose': False,
 'auto_set_spacecharge_mesh': True,
 'archive_path': None,
 'merit_f': None}

# Run CNSGA using processes or threads

In [10]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [11]:
# Create object
X = Xopt(YAML)

# Change some things to make it run faster
X.vocs['constants']['lspch'] = False
X.vocs['constants']['ipart'] = 100
X.vocs['constants']['zstop'] = 0.2
X.results


Loading config from text
Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.


In [12]:
# Run
X.run(executor=executor)

Starting at time 2021-09-27T21:57:44-07:00

    
    
     ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
    ▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
    ▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
    ▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
    ▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
    ░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
      ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
    ░           ░   ░ ░ ░  ░  ░  ░ ░   ░   ░   ▒   
    ░ ░               ░       ░        ░       ░  ░
    ░                                              
    
    
    Continuous Non-dominated Sorting Genetic Algorithm
    Version 0.4.3+178.g392b747.dirty
    
Creating toolbox from vocs.
Created toolbox with 13 variables, 6 constraints, and 2 objectives.
    Using selection algorithm: nsga2
Initializing with a new population, size 8
Maximum generations: 3
____________________________________________________
8 fitness calculations for initial generation
done.
Submitting firs

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Generation 0 completed in 0.02784 minutes
Pop written to  /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp/gen_1.json
Pop written to  /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp/pop_1.json


  0%|          | 0/8 [00:00<?, ?it/s]

Generation 1 completed in 0.01331 minutes
Pop written to  /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp/gen_2.json
Pop written to  /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp/pop_2.json


  0%|          | 0/8 [00:00<?, ?it/s]

Generation 2 completed in 0.01352 minutes
Pop written to  /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp/gen_3.json
Pop written to  /Users/chrisonian/Code/GitHub/xopt/examples/basic/temp/pop_3.json


In [13]:
# Check for errors
X.results['error']

[False, False, False, False, False, False, False, False]

# Write this configuration

In [14]:
X.config['algorithm']['options']['population'] = 'temp/pop_3.json'
X.save('test.yaml')

Config written as YAML to test.yaml


# Run with MPI

In [15]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run -vv --logfile xopt.log test.yaml

Namespace(input_file='test.yaml', logfile='xopt.log', verbose=2)
Parallel execution with 4 workers
Loading from YAML file: test.yaml
Loading config from dict.
Loading config from dict.
Loading config from dict.
Loading config from dict.
/Users/chrisonian/Code/GitHub/xopt/xopt/legacy.py:39: UserWarning: `function` keyword no longer allowed in simulation config, moving to `evaluate`
  warnings.warn('`function` keyword no longer allowed in simulation config, moving to `evaluate`')
Specified both known algorithm `cnsga` and `function`. Using known algorithm function.
Starting at time 2021-09-27T21:58:15-07:00

    
    
     ▄████▄   ███▄    █   ██████   ▄████  ▄▄▄      
    ▒██▀ ▀█   ██ ▀█   █ ▒██    ▒  ██▒ ▀█▒▒████▄    
    ▒▓█    ▄ ▓██  ▀█ ██▒░ ▓██▄   ▒██░▄▄▄░▒██  ▀█▄  
    ▒▓▓▄ ▄██▒▓██▒  ▐▌██▒  ▒   ██▒░▓█  ██▓░██▄▄▄▄██ 
    ▒ ▓███▀ ░▒██░   ▓██░▒██████▒▒░▒▓███▀▒ ▓█   ▓██▒
    ░ ░▒ ▒  ░░ ▒░   ▒ ▒ ▒ ▒▓▒ ▒ ░ ░▒   ▒  ▒▒   ▓▒█░
      ░  ▒   ░ ░░   ░ ▒░░ ░▒  ░ ░  ░   ░   ▒   ▒▒ ░
    ░       

# Dask 

In [ ]:
from dask.distributed import Client
executor = Client()
#executor = Client(processes=True)
executor

In [ ]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [ ]:
X.algorithm['options']['max_generations'] = 4
X.algorithm['options']['population_size'] = 32
X.algorithm['options']['population'] = None
X.results = None
X

In [ ]:
# Run again
X.run(executor=executor)
executor.close()


In [ ]:
X.results['error']

# Plot results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

kx = 'end_sigma_z'
ky = 'end_core_emit_95percent_x'

x = np.array([d[kx] for d in X.results['outputs']])
y = np.array([d[ky] for d in X.results['outputs']])


plt.scatter(x, y)

In [ ]:
X.results['outputs']

# Cleanup

In [ ]:
import shutil
import os
shutil.rmtree('temp/')
os.remove('xopt.log')
#os.chmod('dask-worker-space/',0o777)
shutil.rmtree('dask-worker-space/')
##os.chmod('test.yaml',0o777)
#os.remove('test.yaml')
##os.chmod('NORRAN',0o777)
##os.remove('NORRAN')